In [2]:
import cv2
import numpy as np

yolo_net = cv2.dnn.readNet(r'Downloads/yolov3.weights', r'Downloads/yolov3.cfg')
with open(r'Downloads/coco.names', 'r') as f:
    classes = f.read().strip().split('\n')

# Load OpenCV's pre-trained face detection model (Haar cascades)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to perform detection on each frame
def detect_objects_and_faces(frame):
    height, width = frame.shape[:2]

    # Prepare the image for YOLO
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    yolo_net.setInput(blob)
    layer_names = yolo_net.getUnconnectedOutLayersNames()
    layer_outputs = yolo_net.forward(layer_names)

    # Process YOLO detections
    boxes = []
    confidences = []
    class_ids = []

    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                box = detection[0:4] * np.array([width, height, width, height])
                (centerX, centerY, w, h) = box.astype('int')
                x = int(centerX - w / 2)
                y = int(centerY - h / 2)
                boxes.append([x, y, int(w), int(h)])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, score_threshold=0.5, nms_threshold=0.4)

    # Draw YOLO bounding boxes
    for i in indices.flatten():
        x, y, w, h = boxes[i]
        label = f'{classes[class_ids[i]]}: {confidences[i]:.2f}'
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Detect faces
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

    # Draw face bounding boxes
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, 'Face', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    return frame

# Capture video from file or camera
def process_video(video_source):
    cap = cv2.VideoCapture(video_source)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = detect_objects_and_faces(frame)
        
        # Display the resulting frame
        cv2.imshow('Object and Face Detection', frame)

        # Press 'q' to exit the loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()




In [3]:
# For live camera input (use 0 for the default camera)
camera_index = 0
process_video(camera_index)

In [4]:
# For video file input
video_path = r"C:\Users\srika\Videos\vlc-record-2024-06-01-22h26m33s-Horse_riding_3.mp4-.mp4"
process_video(video_path)